In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import time
import random
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))
from model import OptimalcontrolModel

In [2]:
# Motivation
N_ = 200
dx = 10
du = 20
A = np.random.random((N_,dx,dx))
B = np.random.random((N_,dx,du))

In [3]:
C1 = np.zeros_like(B)
# method 1
for i in range(N_) :
    C1[i] = A[i]@B[i]
# method 2
C2 = A@B    
print(np.sum(np.abs(C1-C2)))

0.0


In [4]:
# computational time
max_iter = 50
list_time1,list_time2 = [],[]
for _ in range(max_iter) :
    tic = time.time()
    for i in range(N_) :
        C1[i] = A[i]@B[i]
    list_time1.append(time.time()-tic)
    tic = time.time()
    C2 = A@B 
    list_time2.append(time.time()-tic)
    assert np.sum(np.abs(C1-C2)) < 1e-10
mean_time1 = sum(list_time1)/len(list_time1)
mean_time2 = sum(list_time2)/len(list_time2)
print(mean_time1)
print(mean_time2)
print(mean_time1/mean_time2)

0.0002598428726196289
5.0115585327148435e-05
5.1848715509039005


## unicycle model (dubin's car)

In [5]:
class unicycle(OptimalcontrolModel):
    def __init__(self,name,ix,iu,linearzation):
        super().__init__(name,ix,iu,linearzation)
        
    def forward(self,x,u,idx=None):
        xdim = np.ndim(x)
        if xdim == 1: # 1 step state & input
            N = 1
            x = np.expand_dims(x,axis=0)
        else :
            N = np.size(x,axis = 0)
        udim = np.ndim(u)
        if udim == 1 :
            u = np.expand_dims(u,axis=0)
     
        # state & input
        x1 = x[:,0]
        x2 = x[:,1]
        x3 = x[:,2]
        
        v = u[:,0]
        w = u[:,1]
        
        # output
        f = np.zeros_like(x)
        f[:,0] = v * np.cos(x3)
        f[:,1] = v * np.sin(x3)
        f[:,2] = w

        return f

    def diff(self,x,u):

        # dimension
        ndim = np.ndim(x)
        if ndim == 1: # 1 step state & input
            N = 1
            x = np.expand_dims(x,axis=0)
            u = np.expand_dims(u,axis=0)
        else :
            N = np.size(x,axis = 0)
        
        # state & input
        x1 = x[:,0]
        x2 = x[:,1]
        x3 = x[:,2]
        
        v = u[:,0]
        w = u[:,1]    
        
        fx = np.zeros((N,self.ix,self.ix))
        fx[:,0,0] = 0.0
        fx[:,0,1] = 0.0
        fx[:,0,2] = - v * np.sin(x3)
        fx[:,1,0] = 0.0
        fx[:,1,1] = 0.0
        fx[:,1,2] = v * np.cos(x3)
        fx[:,2,0] = 0.0
        fx[:,2,1] = 0.0
        fx[:,2,2] = 0.0
        
        fu = np.zeros((N,self.ix,self.iu))
        fu[:,0,0] = np.cos(x3)
        fu[:,0,1] = 0.0
        fu[:,1,0] = np.sin(x3)
        fu[:,1,1] = 0.0
        fu[:,2,0] = 0.0
        fu[:,2,1] = 1.0
        
        return np.squeeze(fx) , np.squeeze(fu)

## Weird nominal trajectory

In [6]:
N = 50
ix = 3
iu = 2

x0 = np.random.random((N+1,ix))
u0 = np.random.random((N+1,iu))
tf0 = 100
print_np(x0)

myModel = unicycle('test',ix,iu,linearzation='analytic') # numeric_central, analytic
N_RK = 10

Type is <class 'numpy.ndarray'>
Shape is (51, 3)


## Discretization

In [7]:
A1,Bm1,Bp1,s1,z1,x_prop1 = myModel.diff_discrete_foh_serial(x0[:N],u0,1/N,tf0,N_RK=N_RK)
A2,Bm2,Bp2,s2,z2,x_prop2 = myModel.diff_discrete_foh(x0[:N],u0,1/N,tf0,N_RK=N_RK)
A3,Bm3,Bp3,s3,z3,x_prop3 = myModel.diff_discrete_foh_variational(x0[:N],u0,1/N,tf0,N_RK=N_RK)

In [8]:
# check the difference
print(np.sum(np.abs(A1-A2)))
print(np.sum(np.abs(A2-A3)))
print('--')
print(np.sum(np.abs(Bm1-Bm2)))
print(np.sum(np.abs(Bm2-Bm3)))
print('--')
print(np.sum(np.abs(Bp1-Bp2)))
print(np.sum(np.abs(Bp2-Bp3)))
print('--')
print(np.sum(np.abs(s1-s2)))
print(np.sum(np.abs(s2-s3)))
print('--')
print(np.sum(np.abs(x_prop1-x_prop2)))
print(np.sum(np.abs(x_prop2-x_prop3)))

0.0
0.0
--
3.4555353484665074e-14
0.0002513301420055573
--
3.005376720240302e-14
0.0002513301420123678
--
2.34783980451736e-16
1.2673802040996557e-06
--
0.0
0.0


In [9]:
# measure time
max_iter = 10
list1 = []
list2 = []
list3 = []
for _ in range(max_iter) :
    tic = time.time()
    A1,Bm1,Bp1,s1,z1,x_prop1 = myModel.diff_discrete_foh_serial(x0[:N],u0,1/N,tf0,N_RK=N_RK)
    list1.append(time.time()-tic)
    tic = time.time()
    A2,Bm2,Bp2,s2,z2,x_prop2 = myModel.diff_discrete_foh(x0[:N],u0,1/N,tf0,N_RK=N_RK)
    list2.append(time.time()-tic)
    tic = time.time()
    A3,Bm3,Bp3,s3,z3,x_prop3 = myModel.diff_discrete_foh_variational(x0[:N],u0,1/N,tf0,N_RK=N_RK)
    list3.append(time.time()-tic)
mean_time1 = sum(list1)/len(list1)
mean_time2 = sum(list2)/len(list2)
mean_time3 = sum(list3)/len(list3)
print(mean_time1)
print(mean_time2)
print(mean_time3)

0.07387139797210693
0.005467557907104492
0.0020673990249633787


In [10]:
print("indirect parallel computation is",mean_time1/mean_time2," times faster than serial")
print("variational approach is",mean_time2/mean_time3," times faster than original")
print("variational approach is",mean_time1/mean_time3," times faster than serial original")

indirect parallel computation is 13.510857905339996  times faster than serial
variational approach is 2.6446553573281975  times faster than original
variational approach is 35.73156274145745  times faster than serial original
